In [1]:
import pandas as pd
import random
from IPython.display import display

# Load dataset
df = pd.read_csv(r"C:\Users\polam\Downloads\AI Diet Planner\data\diet_dataset.csv")

meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

# Explanation templates
templates = {
    "Breakfast": [
        "{food} is a great start for your day; it helps manage {health} and provides slow-releasing energy.",
        "Including {food} for breakfast supports {health} and keeps you energized.",
        "{food} offers essential nutrients for breakfast and is beneficial for {health}."
    ],
    "Lunch": [
        "{food} is perfect for lunch as it balances carbs and protein, supporting {health}.",
        "We chose {food} for lunch to maintain steady energy and help with {health}.",
        "{food} provides essential nutrients for lunch while being suitable for {health} management."
    ],
    "Dinner": [
        "{food} is selected for dinner because it's light, easy to digest, and helps manage {health}.",
        "Dinner with {food} supports your {health} and avoids sugar spikes.",
        "Having {food} at dinner ensures healthy nutrients and helps control {health}."
    ],
    "Snack": [
        "{food} is a healthy snack to manage {health} while keeping you full.",
        "We picked {food} for snack as it is nutritious and supports {health}.",
        "{food} helps control {health} and provides energy between meals."
    ]
}

# User input
age = int(input("Enter your age: "))
gender = input("Enter your gender (M/F/Other): ")
health_condition = input("Enter your health condition (e.g., Diabetes, BP, PCOD, Thyroid, Obesity): ")

# Helper functions
def filter_foods(category, health_condition):
    foods = df[
        (df['Category'] == category) &
        (df['Health Suitability'].str.contains(health_condition, case=False))
    ]
    if len(foods) == 0:
        foods = df[df['Category'] == category]
    return foods

def generate_meal(category, health_condition):
    foods = filter_foods(category, health_condition)
    meal_item = foods.sample(n=1).iloc[0]
    explanation = random.choice(templates[category]).format(food=meal_item['Food Item'], health=health_condition.lower())
    return {
        'Food Item': meal_item['Food Item'],
        'Carbs': meal_item['Carbs'],
        'Protein': meal_item['Protein'],
        'Fat': meal_item['Fats'],
        'Calories': meal_item['Calories'],
        'Explanation': explanation
    }

# Generate 7-day plan
weekly_plan = []
for day in range(1, 8):
    daily_plan = {'Day': f'Day {day}'}
    daily_total_calories = 0
    explanations = []
    
    for meal in meal_types:
        meal_info = generate_meal(meal, health_condition)
        daily_plan[meal] = f"{meal_info['Food Item']} (Carbs: {meal_info['Carbs']}g, Protein: {meal_info['Protein']}g, Fat: {meal_info['Fat']}g, {meal_info['Calories']} kcal)"
        daily_total_calories += meal_info['Calories']
        explanations.append(meal_info['Explanation'])
    
    daily_plan['Total Calories'] = daily_total_calories
    daily_plan['Explanation'] = " ".join(explanations)
    weekly_plan.append(daily_plan)

# Display table
weekly_plan_df = pd.DataFrame(weekly_plan)
pd.set_option('display.max_columns', None)
display(weekly_plan_df.style.set_properties(**{'text-align': 'left'}).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]}]
))

# Save CSV
weekly_plan_df.to_csv('7_day_meal_plan.csv', index=False)
print("\n✅ 7-day meal plan saved as '7_day_meal_plan.csv'")


Enter your age:  20
Enter your gender (M/F/Other):  F
Enter your health condition (e.g., Diabetes, BP, PCOD, Thyroid, Obesity):  Thyroid


,Day,Breakfast,Lunch,Dinner,Snack,Total Calories,Explanation
0,Day 1,"Eggs (Carbs: 1.1g, Protein: 13.0g, Fat: 11.0g, 155 kcal)","Brown Rice (Carbs: 26.0g, Protein: 2.6g, Fat: 1.0g, 123 kcal)","Zucchini (Carbs: 3.1g, Protein: 1.2g, Fat: 0.3g, 17 kcal)","Bell Pepper (Carbs: 6.0g, Protein: 1.0g, Fat: 0.3g, 31 kcal)",326,Eggs offers essential nutrients for breakfast and is beneficial for thyroid. We chose Brown Rice for lunch to maintain steady energy and help with thyroid. Dinner with Zucchini supports your thyroid and avoids sugar spikes. Bell Pepper helps control thyroid and provides energy between meals.
1,Day 2,"Oats (Carbs: 66.0g, Protein: 16.0g, Fat: 7.0g, 389 kcal)","Cabbage (Carbs: 5.8g, Protein: 1.3g, Fat: 0.1g, 25 kcal)","Chicken Breast (Carbs: 0.0g, Protein: 31.0g, Fat: 3.6g, 165 kcal)","Apple (Carbs: 14.0g, Protein: 0.3g, Fat: 0.2g, 52 kcal)",631,"Including Oats for breakfast supports thyroid and keeps you energized. Cabbage is perfect for lunch as it balances carbs and protein, supporting thyroid. Having Chicken Breast at dinner ensures healthy nutrients and helps control thyroid. Apple is a healthy snack to manage thyroid while keeping you full."
2,Day 3,"Oats (Carbs: 66.0g, Protein: 16.0g, Fat: 7.0g, 389 kcal)","Cauliflower (Carbs: 5.0g, Protein: 1.9g, Fat: 0.1g, 25 kcal)","Mushroom (Carbs: 3.3g, Protein: 3.1g, Fat: 0.3g, 22 kcal)","Cucumber (Carbs: 3.6g, Protein: 0.7g, Fat: 0.1g, 16 kcal)",452,Oats is a great start for your day; it helps manage thyroid and provides slow-releasing energy. Cauliflower provides essential nutrients for lunch while being suitable for thyroid management. Dinner with Mushroom supports your thyroid and avoids sugar spikes. We picked Cucumber for snack as it is nutritious and supports thyroid.
3,Day 4,"Eggs (Carbs: 1.1g, Protein: 13.0g, Fat: 11.0g, 155 kcal)","Sweet Potato (Carbs: 26.0g, Protein: 2.0g, Fat: 0.1g, 112 kcal)","Zucchini (Carbs: 3.1g, Protein: 1.2g, Fat: 0.3g, 17 kcal)","Apple (Carbs: 14.0g, Protein: 0.3g, Fat: 0.2g, 52 kcal)",336,"Eggs offers essential nutrients for breakfast and is beneficial for thyroid. Sweet Potato provides essential nutrients for lunch while being suitable for thyroid management. Zucchini is selected for dinner because it's light, easy to digest, and helps manage thyroid. Apple is a healthy snack to manage thyroid while keeping you full."
4,Day 5,"Oats (Carbs: 66.0g, Protein: 16.0g, Fat: 7.0g, 389 kcal)","Brown Rice (Carbs: 26.0g, Protein: 2.6g, Fat: 1.0g, 123 kcal)","Chicken Breast (Carbs: 0.0g, Protein: 31.0g, Fat: 3.6g, 165 kcal)","Cucumber (Carbs: 3.6g, Protein: 0.7g, Fat: 0.1g, 16 kcal)",693,"Oats is a great start for your day; it helps manage thyroid and provides slow-releasing energy. Brown Rice is perfect for lunch as it balances carbs and protein, supporting thyroid. Dinner with Chicken Breast supports your thyroid and avoids sugar spikes. We picked Cucumber for snack as it is nutritious and supports thyroid."
5,Day 6,"Eggs (Carbs: 1.1g, Protein: 13.0g, Fat: 11.0g, 155 kcal)","Cabbage (Carbs: 5.8g, Protein: 1.3g, Fat: 0.1g, 25 kcal)","Chicken Breast (Carbs: 0.0g, Protein: 31.0g, Fat: 3.6g, 165 kcal)","Carrot (Carbs: 10.0g, Protein: 0.9g, Fat: 0.2g, 41 kcal)",386,"Eggs offers essential nutrients for breakfast and is beneficial for thyroid. Cabbage is perfect for lunch as it balances carbs and protein, supporting thyroid. Chicken Breast is selected for dinner because it's light, easy to digest, and helps manage thyroid. Carrot is a healthy snack to manage thyroid while keeping you full."
6,Day 7,"Eggs (Carbs: 1.1g, Protein: 13.0g, Fat: 11.0g, 155 kcal)","Brown Rice (Carbs: 26.0g, Protein: 2.6g, Fat: 1.0g, 123 kcal)","Chicken Breast (Carbs: 0.0g, Protein: 31.0g, Fat: 3.6g, 165 kcal)","Carrot (Carbs: 10.0g, Protein: 0.9g, Fat: 0.2g, 41 kcal)",484,"Eggs is a great start for your day; it helps manage thyroid and provides slow-releasing energy. Brown Rice is perfect for lunch as it balances carbs and protein, supporting thyroid. Dinner with Chicken Bre


✅ 7-day meal plan saved as '7_day_meal_plan.csv'


In [2]:
import pandas as pd
import random
import re
from IPython.display import display, clear_output

# Load dataset
df = pd.read_csv(r"C:\Users\polam\Downloads\AI Diet Planner\data\diet_dataset.csv")
meal_types = ['Breakfast', 'Lunch', 'Dinner', 'Snack']

# Explanation templates
templates = {
    "Breakfast": [
        "{food} is a great start for your day; it helps manage {health} and provides slow-releasing energy.",
        "Including {food} for breakfast supports {health} and keeps you energized.",
        "{food} offers essential nutrients for breakfast and is beneficial for {health}."
    ],
    "Lunch": [
        "{food} is perfect for lunch as it balances carbs and protein, supporting {health}.",
        "We chose {food} for lunch to maintain steady energy and help with {health}.",
        "{food} provides essential nutrients for lunch while being suitable for {health} management."
    ],
    "Dinner": [
        "{food} is selected for dinner because it's light, easy to digest, and helps manage {health}.",
        "Dinner with {food} supports your {health} and avoids sugar spikes.",
        "Having {food} at dinner ensures healthy nutrients and helps control {health}."
    ],
    "Snack": [
        "{food} is a healthy snack to manage {health} while keeping you full.",
        "We picked {food} for snack as it is nutritious and supports {health}.",
        "{food} helps control {health} and provides energy between meals."
    ]
}

# Helper functions
def filter_foods(category, health_condition):
    foods = df[
        (df['Category'] == category) &
        (df['Health Suitability'].str.contains(health_condition, case=False))
    ]
    if len(foods) == 0:
        foods = df[df['Category'] == category]
    return foods

def generate_meal(category, health_condition):
    foods = filter_foods(category, health_condition)
    meal_item = foods.sample(n=1).iloc[0]
    explanation = random.choice(templates[category]).format(food=meal_item['Food Item'], health=health_condition.lower())
    return {
        'Food Item': meal_item['Food Item'],
        'Carbs': meal_item['Carbs'],
        'Protein': meal_item['Protein'],
        'Fat': meal_item['Fats'],
        'Calories': meal_item['Calories'],
        'Explanation': explanation
    }

def generate_weekly_plan(age, gender, health_condition):
    weekly_plan = []
    for day in range(1, 8):
        daily_plan = {'Day': f'Day {day}'}
        daily_total_calories = 0
        explanations = []
        for meal in meal_types:
            meal_info = generate_meal(meal, health_condition)
            daily_plan[meal] = f"{meal_info['Food Item']} (Carbs: {meal_info['Carbs']}g, Protein: {meal_info['Protein']}g, Fat: {meal_info['Fat']}g, {meal_info['Calories']} kcal)"
            daily_total_calories += meal_info['Calories']
            explanations.append(meal_info['Explanation'])
        daily_plan['Total Calories'] = daily_total_calories
        daily_plan['Explanation'] = " ".join(explanations)
        weekly_plan.append(daily_plan)
    return pd.DataFrame(weekly_plan)

# ----------------- Chatbot Loop -----------------
print("🤖 Welcome to AI Health-Aware Diet Planner Chatbot 🥗\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Goodbye! Stay healthy! 🥗")
        break
    
    # Parse age
    age_match = re.search(r'\b(\d{1,2})\s*years?\b', user_input)
    age = int(age_match.group(1)) if age_match else 25  # default 25
    
    # Parse gender
    if "female" in user_input.lower():
        gender = "F"
    elif "male" in user_input.lower():
        gender = "M"
    else:
        gender = "Other"
    
    # Parse health condition (simplified)
    health_conditions = ["Diabetes", "BP", "PCOD", "Thyroid", "Obesity"]
    health_condition = next((h for h in health_conditions if h.lower() in user_input.lower()), "Diabetes")
    
    # Generate plan
    weekly_plan_df = generate_weekly_plan(age, gender, health_condition)
    
    # Display
    clear_output(wait=True)
    print("🤖 Here is your 7-Day AI-Generated Meal Plan 🥗\n")
    display(weekly_plan_df)
    print("\nType 'exit' or 'quit' to end the chat.")

🤖 Here is your 7-Day AI-Generated Meal Plan 🥗



,Day,Breakfast,Lunch,Dinner,Snack,Total Calories,Explanation
0,Day 1,"Yogurt (Carbs: 3.6g, Protein: 10.0g, Fat: 0.4g...","Cottage Cheese (Low-fat) (Carbs: 3.4g, Protein...","Broccoli (Carbs: 7.0g, Protein: 2.8g, Fat: 0.4...","Cucumber (Carbs: 3.6g, Protein: 0.7g, Fat: 0.1...",207,Including Yogurt for breakfast supports diabet...
1,Day 2,"Ragi (Carbs: 72.0g, Protein: 7.3g, Fat: 1.4g, ...","Chickpeas (Carbs: 27.0g, Protein: 9.0g, Fat: 2...","Zucchini (Carbs: 3.1g, Protein: 1.2g, Fat: 0.3...","Watermelon (Carbs: 8.0g, Protein: 0.6g, Fat: 0...",547,Ragi is a great start for your day; it helps m...
2,Day 3,"Banana (Carbs: 23.0g, Protein: 1.0g, Fat: 0.3g...","Lentils (Carbs: 20.0g, Protein: 9.0g, Fat: 0.4...","Spinach (Carbs: 3.6g, Protein: 2.9g, Fat: 0.4g...","Apple (Carbs: 14.0g, Protein: 0.3g, Fat: 0.2g,...",280,Banana offers essential nutrients for breakfas...
3,Day 4,"Banana (Carbs: 23.0g, Protein: 1.0g, Fat: 0.3g...","Cauliflower (Carbs: 5.0g, Protein: 1.9g, Fat: ...","Zucchini (Carbs: 3.1g, Protein: 1.2g, Fat: 0.3...","Watermelon (Carbs: 8.0g, Protein: 0.6g, Fat: 0...",161,Banana is a great start for your day; it helps...
4,Day 5,"Millets (Carbs: 73.0g, Protein: 11.0g, Fat: 4....","Methi (Fenugreek) (Carbs: 7.0g, Protein: 4.0g,...","Broccoli (Carbs: 7.0g, Protein: 2.8g, Fat: 0.4...","Blueberries (Carbs: 14.0g, Protein: 0.7g, Fat:...",518,Millets is a great start for your day; it help...
5,Day 6,"Banana (Carbs: 23.0g, Protein: 1.0g, Fat: 0.3g...","Methi (Fenugreek) (Carbs: 7.0g, Protein: 4.0g,...","Zucchini (Carbs: 3.1g, Protein: 1.2g, Fat: 0.3...","Walnuts (Carbs: 14.0g, Protein: 15.0g, Fat: 65...",809,Banana offers essential nutrients for breakfas...
6,Day 7,"Millets (Carbs: 73.0g, Protein: 11.0g, Fat: 4....","Cabbage (Carbs: 5.8g, Protein: 1.3g, Fat: 0.1g...","Pumpkin (Carbs: 6.5g, Protein: 1.0g, Fat: 0.1g...","Flax Seeds (Carbs: 29.0g, Protein: 18.0g, Fat:...",963,Millets offers essential nutrients for breakfa...



Type 'exit' or 'quit' to end the chat.


You:  exit


🤖 Goodbye! Stay healthy! 🥗


In [3]:
# ----------------- Conversational AI Diet Chatbot -----------------
import re
from IPython.display import display, clear_output
import random

# Keep user state
user_state = {
    "age": 25,
    "gender": "Other",
    "health_condition": "Diabetes",
    "diet_type": "any",  # can be 'vegetarian', 'non-vegetarian', or 'any'
    "weekly_plan": None
}

print("🤖 Welcome to the AI Health-Aware Diet Planner Chatbot 🥗")
print("You can type requests like:")
print("- 'Schedule a weekly meal plan for me, I am 25 years old female with BP.'")
print("- 'Change Day 3 lunch to something vegetarian.'")
print("- 'Show explanation for Day 1 dinner.'")
print("Type 'exit' or 'quit' to end the chat.\n")

# ----------------- Chat Loop -----------------
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Goodbye! Stay healthy! 🥗")
        break
    
    clear_output(wait=True)
    
    # --- Step 1: Update user info if provided ---
    age_match = re.search(r'\b(\d{1,2})\s*years?\b', user_input)
    if age_match:
        user_state["age"] = int(age_match.group(1))
    
    if "female" in user_input.lower():
        user_state["gender"] = "F"
    elif "male" in user_input.lower():
        user_state["gender"] = "M"
    
    health_conditions = ["Diabetes", "BP", "PCOD", "Thyroid", "Obesity"]
    for h in health_conditions:
        if h.lower() in user_input.lower():
            user_state["health_condition"] = h
    
    # Detect diet type change
    if "vegetarian" in user_input.lower():
        user_state["diet_type"] = "vegetarian"
    elif "non-vegetarian" in user_input.lower():
        user_state["diet_type"] = "non-vegetarian"
    
    # --- Step 2: Handle weekly plan generation ---
    if "week" in user_input.lower() or "meal plan" in user_input.lower():
        user_state["weekly_plan"] = generate_weekly_plan(
            user_state["age"], user_state["gender"], user_state["health_condition"]
        )
        print(f"🤖 7-Day AI-Generated Meal Plan for {user_state['health_condition']}:\n")
        display(user_state["weekly_plan"])
        print("\nYou can modify any meal or ask for explanations.\n")
        continue
    
    # --- Step 3: Modify specific meals ---
    day_match = re.search(r'Day\s*(\d)', user_input, re.IGNORECASE)
    meal_match = re.search(r'(breakfast|lunch|dinner|snack)', user_input, re.IGNORECASE)
    
    if day_match and meal_match and "change" in user_input.lower():
        day = int(day_match.group(1))
        meal = meal_match.group(1).capitalize()
        print(f"🤖 Updating {meal} for Day {day}...")
        new_meal_info = generate_meal(meal, user_state["health_condition"])
        # Update weekly_plan DataFrame
        if user_state["weekly_plan"] is not None:
            user_state["weekly_plan"].at[day-1, meal] = f"{new_meal_info['Food Item']} (Carbs: {new_meal_info['Carbs']}g, Protein: {new_meal_info['Protein']}g, Fat: {new_meal_info['Fat']}g, {new_meal_info['Calories']} kcal)"
            user_state["weekly_plan"].at[day-1, 'Explanation'] += f" Updated {meal}: {new_meal_info['Explanation']}"
            display(user_state["weekly_plan"])
        continue
    
    # --- Step 4: Show explanation for specific meal ---
    if day_match and meal_match and "explanation" in user_input.lower():
        day = int(day_match.group(1))
        meal = meal_match.group(1).capitalize()
        if user_state["weekly_plan"] is not None:
            exp_text = user_state["weekly_plan"].at[day-1, 'Explanation']
            print(f"🤖 Explanation for Day {day} {meal}:\n{exp_text}")
        continue
    
    # --- Step 5: Default response ---
    print("🤖 I didn’t fully understand. You can ask me to:")
    print("- Generate a weekly meal plan")
    print("- Change a specific meal (e.g., 'Change Day 3 lunch')")
    print("- Show explanation for a meal (e.g., 'Explain Day 1 breakfast')")

🤖 Updating Breakfast for Day 2...


,Day,Breakfast,Lunch,Dinner,Snack,Total Calories,Explanation
0,Day 1,"Yogurt (Carbs: 3.6g, Protein: 10.0g, Fat: 0.4g...","Paneer (Carbs: 1.2g, Protein: 18.0g, Fat: 20.0...","Chicken Breast (Carbs: 0.0g, Protein: 31.0g, F...","Flax Seeds (Carbs: 29.0g, Protein: 18.0g, Fat:...",1023,Yogurt offers essential nutrients for breakfas...
1,Day 2,"Banana (Carbs: 23.0g, Protein: 1.0g, Fat: 0.3g...","Lentils (Carbs: 20.0g, Protein: 9.0g, Fat: 0.4...","Mushroom (Carbs: 3.3g, Protein: 3.1g, Fat: 0.3...","Cucumber (Carbs: 3.6g, Protein: 0.7g, Fat: 0.1...",543,Including Oats for breakfast supports bp and k...
2,Day 3,"Eggs (Carbs: 1.1g, Protein: 13.0g, Fat: 11.0g,...","Cottage Cheese (Low-fat) (Carbs: 3.4g, Protein...","Fish (Salmon) (Carbs: 0.0g, Protein: 20.0g, Fa...","Pomegranate (Carbs: 19.0g, Protein: 1.7g, Fat:...",544,Eggs offers essential nutrients for breakfast ...
3,Day 4,"Yogurt (Carbs: 3.6g, Protein: 10.0g, Fat: 0.4g...","Kidney Beans (Carbs: 22.0g, Protein: 8.7g, Fat...","Chicken Breast (Carbs: 0.0g, Protein: 31.0g, F...","Beetroot (Carbs: 10.0g, Protein: 1.6g, Fat: 0....",394,Yogurt offers essential nutrients for breakfas...
4,Day 5,"Banana (Carbs: 23.0g, Protein: 1.0g, Fat: 0.3g...","Methi (Fenugreek) (Carbs: 7.0g, Protein: 4.0g,...","Pumpkin (Carbs: 6.5g, Protein: 1.0g, Fat: 0.1g...","Blueberries (Carbs: 14.0g, Protein: 0.7g, Fat:...",221,Including Banana for breakfast supports bp and...
5,Day 6,"Eggs (Carbs: 1.1g, Protein: 13.0g, Fat: 11.0g,...","Green Gram (Moong) (Carbs: 63.0g, Protein: 24....","Zucchini (Carbs: 3.1g, Protein: 1.2g, Fat: 0.3...","Watermelon (Carbs: 8.0g, Protein: 0.6g, Fat: 0...",549,Eggs offers essential nutrients for breakfast ...
6,Day 7,"Millets (Carbs: 73.0g, Protein: 11.0g, Fat: 4....","Green Gram (Moong) (Carbs: 63.0g, Protein: 24....","Pumpkin (Carbs: 6.5g, Protein: 1.0g, Fat: 0.1g...","Cucumber (Carbs: 3.6g, Protein: 0.7g, Fat: 0.1...",767,Including Millets for breakfast supports bp an...


You:  exit


🤖 Goodbye! Stay healthy! 🥗
